# Destinations Report

Resumen ejecutivo:
    - Los calidad de los datos es aceptable
    - Pero son muy pocos: ¿ Será la manera correcta de extraerlos del portal: Reports > DestinationReports > zona: Chile > Generar ?

# ETL

In [1]:
from pathlib import Path
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import pickle
import re

In [2]:
directory='Destination'
file_list = sorted(Path(Path.cwd(),directory).glob('[0-9]*ShipTracks*.csv'))
len(file_list),' files found!'

(2, ' files found!')

In [3]:
afile = file_list[0]
df1 = pd.read_csv(afile)
df1[:4]

,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
0,AGUNSA CAPELLA,9469209.0,725001506,CBCW,0.0,230.0,TALCAHUANO,05/20/2023 2:00:00 PM UTC,4.9,-36.671467,-73.068250,5/24/2023 6:12:01 AM,5/24/2023 2:12:01 AM,Not-available,Chile,"Talcahuano, CHI",NSC Shipmanagement Chile Ltda,2147.0,78.02
1,ALKHAIRAT 9,9152806.0,438031042,JYAW9,0.6,118.2,"PUERTO MONTT,CHILE",06/21/2023 5:00:00 PM UTC,7.8,-41.507087,-72.950242,5/24/2023 6:12:14 AM,5/24/2023 2:12:14 AM,Cargo ship,Chile,"Calbuco, CHI",Coral Technical Services,10094.0,127.90
2,ANTARCTIC ENDEAVOUR,8717453.0,725001619,CA5744,1.3,252.8,PUNTA ARENAS CHILE,NaN,5.0,-63.321717,-59.827883,5/24/2023 6:01:57 AM,5/24/2023 2:01:57 AM,Fishing,NaN,NaN,Deris SA,600.0,49.98
3,ANTOFAGASTA,9629718.0,725001411,CBAZ,0.2,261.3,TALCAHUANO,05/23/2023 5:00:00 AM UTC,8.3,-36.699837,-73.037802,5/24/2023 6:12:46 AM,5/24/2023 2:12:46 AM,Tanker,Chile,"Talcahuano, CHI",NSC Shipmanagement Chile Ltda,49996.0,183.00


In [4]:
detypes = { c:df1[c].dtype for c in df1.columns}
detypes

{'Name': dtype('O'),
 'IMO': dtype('float64'),
 'MMSI': dtype('int64'),
 'Callsign': dtype('O'),
 'Speed': dtype('float64'),
 'Course': dtype('float64'),
 'Destination': dtype('O'),
 'ETA': dtype('O'),
 'Draught': dtype('float64'),
 'Lat': dtype('float64'),
 'Lon': dtype('float64'),
 'UTC Time': dtype('O'),
 'Local Time': dtype('O'),
 'Ship Type': dtype('O'),
 'Zone': dtype('O'),
 'Location': dtype('O'),
 'Operator': dtype('O'),
 'Dead Weight': dtype('float64'),
 'LOA': dtype('float64')}

## Extraccion

In [5]:
pickle_file = Path(directory,'df.p')

df = pd.DataFrame( columns=['filedate',*df1.columns] )

if Path(pickle_file).is_file():
    with open(pickle_file,'rb') as afile:
        df = pickle.load(afile)
else:
    for afile in file_list:
        tmp = pd.read_csv(afile)
        tmp.insert(0, 'filedate', afile.stem.replace(' DestinationReport ShipTracks Web',''))
        df = pd.concat((df,tmp))
    with open(pickle_file,'wb') as afile:
        pickle.dump(df,afile)

## Limpieza

In [6]:
### retrospectiva filas con utc != local
df.iloc[[ 5,  9, 39, 56, 65, 69, 73]]

,filedate,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
5,20230523,AQUAPUELCHE,9778727.0,636017857,D5NA5,0.1,295.0,QUINTERO CHILE,02/23/2023 02:00:00 AM,8.4,-32.754595,-71.511135,2/23/2023 2:04:20 PM,2/23/2023 10:04:20 AM,Tanker,Chile,"Puerto Ventanas, CHI",NaN,NaN,NaN
9,20230523,BRIO,9655913.0,725001963,CBBR,0.0,266.4,CL-TALCAHUANO,05/06/2023 22:00:00,8.1,-36.694688,-73.097082,5/10/2023 12:44:44 AM,5/9/2023 8:44:44 PM,Tanker,Chile,"Talcahuano, CHI",NSC Shipmanagement Chile Ltda,49990.0,183.31
39,20230523,MANJIN,NaN,440192680,620MANJ,16.4,222.0,C/CHILE,01/01/1970 00:00:00 AM,NaN,34.207800,127.186228,5/21/2023 12:04:49 AM,5/20/2023 8:04:49 PM,Fishing,NaN,"42 NM SW of Yeosu, KOR",NaN,NaN,NaN
1,20230525,AQUAPUELCHE,9778727.0,636017857,D5NA5,0.1,295.0,QUINTERO CHILE,02/23/2023 02:00:00 AM,8.4,-32.754595,-71.511135,2/23/2023 2:04:20 PM,2/23/2023 10:04:20 AM,Tanker,Chile,"Puerto Ventanas, CHI",NaN,NaN,NaN
10,20230525,KINGSTAR,8916009.0,440203000,6KCD4,5.9,96.1,CHILE,01/20/2024 1:00:00 PM UTC,4.8,-45.443947,-59.607285,5/25/2023 3:12:32 PM,5/25/2023 11:12:32 AM,Fishing,NaN,South Atlantic Ocean,NaN,NaN,NaN
14,20230525,MANJIN,NaN,440192680,620MANJ,16.4,222.0,C/CHILE,01/01/1970 00:00:00 AM,NaN,34.207800,127.186228,5/21/2023 12:04:49 AM,5/20/2023 8:04:49 PM,Fishing,NaN,"42 NM SW of Yeosu, KOR",NaN,NaN,NaN
18,20230525,RENAISSANCE,NaN,316038749,CFA3166,0.0,192.3,#- CHILE,11/30/2021 00:00:00 AM,NaN,48.451863,-123.294160,5/25/2023 3:01:17 AM,5/24/2023 11:01:17 PM,Sailing,NaN,"Ogden Point, CAN",NaN,NaN,NaN


## Transformacion

In [29]:
{ c:df1[c].dtype for c in df1.columns}

{'Name': dtype('O'),
 'IMO': dtype('float64'),
 'MMSI': dtype('int64'),
 'Callsign': dtype('O'),
 'Speed': dtype('float64'),
 'Course': dtype('float64'),
 'Destination': dtype('O'),
 'ETA': dtype('O'),
 'Draught': dtype('float64'),
 'Lat': dtype('float64'),
 'Lon': dtype('float64'),
 'UTC Time': dtype('O'),
 'Local Time': dtype('O'),
 'Ship Type': dtype('O'),
 'Zone': dtype('O'),
 'Location': dtype('O'),
 'Operator': dtype('O'),
 'Dead Weight': dtype('float64'),
 'LOA': dtype('float64')}

In [8]:
df.ETA.isna().sum()/len(df)

0.25

In [9]:
#df.ETA = df.ETA.apply(lambda x : re.sub(r'\s*(AM|PM)\s*', '', str(x)).replace('UTC',''))
df.ETA = df.ETA.apply(lambda x : str(x).replace('UTC',''))
df['UTC Time'] = df['UTC Time'].apply(lambda x : str(x).replace('UTC',''))
df['Local Time'] = df['Local Time'].apply(lambda x : str(x).replace('UTC',''))

In [10]:
# df.ETA.fillna(pd.to_datetime(0, unit='s'), inplace=True)

In [11]:
df.ETA = pd.to_datetime(df.ETA, errors='coerce')
df['UTC Time'] = pd.to_datetime(df['UTC Time'])
df['Local Time'] = pd.to_datetime(df['Local Time'])
df.filedate = pd.to_datetime(df.filedate)

C:\Users\FernandoBadilla\AppData\Local\Temp\ipykernel_2784\2288412679.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.ETA = pd.to_datetime(df.ETA, errors='coerce')


In [12]:
df.ETA

0    2023-05-20 14:00:00
1    2023-06-21 17:00:00
2                    NaT
3    2023-05-23 05:00:00
4    2023-05-19 08:00:00
             ...        
16   2023-12-05 09:00:00
17                   NaT
18   2021-11-30 00:00:00
19   2023-05-25 11:30:00
20   1970-01-01 00:00:00
Name: ETA, Length: 76, dtype: datetime64[ns]

In [13]:
df[:4]

,filedate,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
0,2023-05-23,AGUNSA CAPELLA,9469209.0,725001506,CBCW,0.0,230.0,TALCAHUANO,2023-05-20 14:00:00,4.9,-36.671467,-73.068250,2023-05-24 06:12:01,2023-05-24 02:12:01,Not-available,Chile,"Talcahuano, CHI",NSC Shipmanagement Chile Ltda,2147.0,78.02
1,2023-05-23,ALKHAIRAT 9,9152806.0,438031042,JYAW9,0.6,118.2,"PUERTO MONTT,CHILE",2023-06-21 17:00:00,7.8,-41.507087,-72.950242,2023-05-24 06:12:14,2023-05-24 02:12:14,Cargo ship,Chile,"Calbuco, CHI",Coral Technical Services,10094.0,127.90
2,2023-05-23,ANTARCTIC ENDEAVOUR,8717453.0,725001619,CA5744,1.3,252.8,PUNTA ARENAS CHILE,NaT,5.0,-63.321717,-59.827883,2023-05-24 06:01:57,2023-05-24 02:01:57,Fishing,NaN,NaN,Deris SA,600.0,49.98
3,2023-05-23,ANTOFAGASTA,9629718.0,725001411,CBAZ,0.2,261.3,TALCAHUANO,2023-05-23 05:00:00,8.3,-36.699837,-73.037802,2023-05-24 06:12:46,2023-05-24 02:12:46,Tanker,Chile,"Talcahuano, CHI",NSC Shipmanagement Chile Ltda,49996.0,183.00


## datetime raros

In [15]:
np.all( df['UTC Time']==df['Local Time'] )

False

In [16]:
np.where( ((df['UTC Time'] - df['Local Time']) != pd.Timedelta(hours=4)))

(array([ 5,  9, 39, 56, 65, 69, 73], dtype=int64),)

# Panorama llegadas en los proximos dias

In [24]:
days = 30

In [25]:
dfa = df[ (df.ETA < pd.Timestamp.now() + pd.Timedelta(days=days)) & (df.ETA > pd.Timestamp.now()) ]

In [26]:
dfa = dfa.sort_values('ETA')

In [27]:
dfa

,filedate,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
30,2023-05-23,HELLAS DYNASTY,9869241.0,538008424,V7A2143,14.4,185.9,QUINTERO CHILE,2023-05-26 01:00:00,11.7,-23.987718,-70.917573,2023-05-24 05:45:44,2023-05-24 01:45:44,Tanker,Chile,"28 NM SW of Caleta Coloso, CHI",Latsco Marine Management Inc,51116.0,229.940
7,2023-05-25,HELLAS DYNASTY,9869241.0,538008424,V7A2143,7.9,185.7,QUINTERO CHILE,2023-05-26 09:00:00,11.8,-30.820023,-72.186888,2023-05-25 10:15:17,2023-05-25 06:15:17,Tanker,Chile,Pacific South Americas,Latsco Marine Management Inc,51116.0,229.940
15,2023-05-25,MARCILIO DIAS,9453858.0,710024830,PPPT,12.5,198.6,CHILE-SANVICENT,2023-05-30 20:00:00,16.4,-53.621750,-70.834282,2023-05-25 09:41:54,2023-05-25 05:41:54,Tanker,Chile,"25 NM S of Lenadura, CHI",TRANSPETRO,156548.0,274.200
43,2023-05-23,NIGHTSKY,9847243.0,311000894,C6EH3,11.7,123.9,"HUASCO,CHILE",2023-05-31 11:30:00,11.8,-8.343568,-106.081580,2023-05-24 05:53:13,2023-05-24 01:53:13,Cargo ship,NaN,Pacific South Americas,Oldendorff Carriers GmbH & Co,183017.0,291.990
2,2023-05-25,DAQING KNUTSEN,9914656.0,259126000,LANG8,11.7,214.4,CHILE FOR ORDERS,2023-06-01 00:00:00,16.6,-47.474963,-61.322872,2023-05-25 10:07:56,2023-05-25 06:07:56,Tanker,NaN,South Atlantic Ocean,Knutsen NYK Offshore Tankers,154955.0,278.909
4,2023-05-25,ES LEADER,9781009.0,352359000,3EUK3,9.8,139.6,PUERTO MONTT-CHILE,2023-06-18 05:00:00,10.3,-11.014963,167.351530,2023-05-25 10:13:53,2023-05-25 06:13:53,Cargo ship,NaN,Pacific South Asia,Erasmus Corp,64012.0,199.900
1,2023-05-23,ALKHAIRAT 9,9152806.0,438031042,JYAW9,0.6,118.2,"PUERTO MONTT,CHILE",2023-06-21 17:00:00,7.8,-41.507087,-72.950242,2023-05-24 06:12:14,2023-05-24 02:12:14,Cargo ship,Chile,"Calbuco, CHI",Coral Technical Services,10094.0,127.900
0,2023-05-25,ALKHAIRAT 9,9152806.0,438031042,JYAW9,0.1,351.0,"PUERTO MONTT,CHILE",2023-06-21 17:00:00,7.8,-41.503277,-72.947545,2023-05-25 10:25:19,2023-05-25 06:25:19,Cargo ship,Chile,"Calbuco, CHI",Coral Technical Services,10094.0,127.900


## agrupado por nombre

In [28]:
for name, group in df[ (df.ETA < pd.Timestamp.now() + pd.Timedelta(days=days)) & (df.ETA > pd.Timestamp.now()) ].groupby('Name'):
    print(name)
    display(group)

ALKHAIRAT 9


,filedate,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
1,2023-05-23,ALKHAIRAT 9,9152806.0,438031042,JYAW9,0.6,118.2,"PUERTO MONTT,CHILE",2023-06-21 17:00:00,7.8,-41.507087,-72.950242,2023-05-24 06:12:14,2023-05-24 02:12:14,Cargo ship,Chile,"Calbuco, CHI",Coral Technical Services,10094.0,127.9
0,2023-05-25,ALKHAIRAT 9,9152806.0,438031042,JYAW9,0.1,351.0,"PUERTO MONTT,CHILE",2023-06-21 17:00:00,7.8,-41.503277,-72.947545,2023-05-25 10:25:19,2023-05-25 06:25:19,Cargo ship,Chile,"Calbuco, CHI",Coral Technical Services,10094.0,127.9


DAQING KNUTSEN


,filedate,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
2,2023-05-25,DAQING KNUTSEN,9914656.0,259126000,LANG8,11.7,214.4,CHILE FOR ORDERS,2023-06-01,16.6,-47.474963,-61.322872,2023-05-25 10:07:56,2023-05-25 06:07:56,Tanker,NaN,South Atlantic Ocean,Knutsen NYK Offshore Tankers,154955.0,278.909


ES LEADER


...

,filedate,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
30,2023-05-23,HELLAS DYNASTY,9869241.0,538008424,V7A2143,14.4,185.9,QUINTERO CHILE,2023-05-26 01:00:00,11.7,-23.987718,-70.917573,2023-05-24 05:45:44,2023-05-24 01:45:44,Tanker,Chile,"28 NM SW of Caleta Coloso, CHI",Latsco Marine Management Inc,51116.0,229.94
7,2023-05-25,HELLAS DYNASTY,9869241.0,538008424,V7A2143,7.9,185.7,QUINTERO CHILE,2023-05-26 09:00:00,11.8,-30.820023,-72.186888,2023-05-25 10:15:17,2023-05-25 06:15:17,Tanker,Chile,Pacific South Americas,Latsco Marine Management Inc,51116.0,229.94


MARCILIO DIAS


,filedate,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
15,2023-05-25,MARCILIO DIAS,9453858.0,710024830,PPPT,12.5,198.6,CHILE-SANVICENT,2023-05-30 20:00:00,16.4,-53.62175,-70.834282,2023-05-25 09:41:54,2023-05-25 05:41:54,Tanker,Chile,"25 NM S of Lenadura, CHI",TRANSPETRO,156548.0,274.2


NIGHTSKY


,filedate,Name,IMO,MMSI,Callsign,Speed,Course,Destination,ETA,Draught,Lat,Lon,UTC Time,Local Time,Ship Type,Zone,Location,Operator,Dead Weight,LOA
43,2023-05-23,NIGHTSKY,9847243.0,311000894,C6EH3,11.7,123.9,"HUASCO,CHILE",2023-05-31 11:30:00,11.8,-8.343568,-106.08158,2023-05-24 05:53:13,2023-05-24 01:53:13,Cargo ship,NaN,Pacific South Americas,Oldendorff Carriers GmbH & Co,183017.0,291.99
